In [34]:
import pandas as pd
import numpy as np
import os, pickle
import warnings
import datetime
warnings.filterwarnings('ignore')

In [35]:
path_to_data = r"../../data/raw/stock prices"

In [36]:
# Get the list of all csv files in path_to_data and all subfolders
csv_files = []
for root, dirs, files in os.walk(path_to_data):
    for f in files:
        if f.endswith('.csv'):
            csv_files.append(os.path.join(root, f))

In [37]:
# Loop through the files, limiting columns and appending airline tickers to a df
for file in csv_files:
    ticker = file.split('_')[0].split('\\')[-1].upper()  # Extract ticker from filename

    if file == csv_files[0]:
        df_main = pd.read_csv(file)
        df_main['ticker'] = ticker
    else:
        df_temp = pd.read_csv(file)
        df_temp['ticker'] = ticker
        df_main = pd.concat([df_main, df_temp], ignore_index=True)
df_main['ticker'].value_counts()

ticker
ALGT    80000
ALK     80000
DAL     80000
UAL     80000
ITA     80000
LUV     80000
JBLU    80000
IYT     79869
AAL     74806
JETS    62745
BNO     60000
Name: count, dtype: int64

In [38]:
# Drop duplicates rows
df_main = df_main.drop_duplicates()
df_main = df_main.dropna()

In [39]:
# Convert 'Time' column to datetime format
df_main['Time'] = pd.to_datetime(df_main['Time'], format='%Y-%m-%d %H:%M')
df_main['%Chg'] = df_main['%Chg'].str.replace('%', '').astype(float)
df_main

,Time,Open,High,Low,Last,Change,%Chg,Volume,ticker
0,2022-05-03 11:15:00,18.6900,18.770,18.6650,18.7450,0.0550,0.29,844892.0,AAL
1,2022-05-03 11:30:00,18.7450,18.855,18.7350,18.8350,0.0900,0.48,790483.0,AAL
2,2022-05-03 11:45:00,18.8400,18.850,18.7400,18.7761,-0.0589,-0.31,600037.0,AAL
3,2022-05-03 12:00:00,18.7703,18.810,18.6540,18.7272,-0.0489,-0.26,738723.0,AAL
4,2022-05-03 12:15:00,18.7300,18.820,18.7300,18.7400,0.0128,0.07,582906.0,AAL
...,...,...,...,...,...,...,...,...,...
837395,2022-05-03 10:00:00,50.5000,50.750,49.8250,49.9700,-0.5300,-1.05,472374.0,UAL
837396,2022-05-03 10:15:00,49.9700,50.270,49.6500,50.2400,0.2700,0.54,388933.0,UAL
837397,2022-05-03 10:30:00,50.2400,50.550,50.0800,50.1600,-0.0800,-0.16,346036.0,UAL
837398,2022-05-03 10:45:00,50.1700,50.520,50.0301,50.4450,0.2850,0.57,319102.0,UAL


In [40]:
# Finance variables
df_main['High-Low']   =  df_main['High'] - df_main['Low']
df_main['High-Low%']  = (df_main['High'] / df_main['Open']) - 1
df_main['Last-Open']  =  df_main['Open'] - df_main['Last']
df_main['Last-Open%'] = (df_main['Open'] / df_main['Last']) - 1

df_main.sort_values(by=['ticker', 'Time'], inplace=True)

# 10-period rolling variance
df_main['Roll_SD_Last_10']   = df_main.groupby('ticker')['Last'].rolling(window=10).std().reset_index(level=0, drop=True)
df_main['Roll_SDSD_Last_10'] = df_main.groupby('ticker')['Roll_SD_Last_10'].rolling(window=10).std().reset_index(level=0, drop=True)

df_main['Roll_SD_Volume_10']   = df_main.groupby('ticker')['Volume'].rolling(window=10).std().reset_index(level=0, drop=True)
df_main['Roll_SDSD_Volume_10'] = df_main.groupby('ticker')['Roll_SD_Volume_10'].rolling(window=10).std().reset_index(level=0, drop=True)

for i in ['Last', 'Volume', 'High-Low', 'Last-Open', 'Roll_SD_Last_10', 'Roll_SDSD_Last_10', 'Roll_SD_Volume_10', 'Roll_SDSD_Volume_10']:
    df_main[f'Change_{i}'] = df_main.groupby('ticker')[i].diff()

In [41]:
df_main['ti']=df_main['Time'].dt.time
df_main.sort_values(by=['ticker', 'Time'], inplace=True)

# More efficient approach for lagged variables
for day_lag in range(1, 11):
    df_main[f'Change_Volume_Day_lag{day_lag:02d}'] = None
    
    for time_val in df_main['ti'].unique():
        mask = df_main['ti'] == time_val
        subset = df_main[mask].copy()
        subset = subset.sort_values(['ticker', 'Time'])
        
        # Calculate lag for each ticker separately
        lagged_values = subset.groupby('ticker')['Volume'].diff().shift(day_lag)
        df_main.loc[mask, f'Change_Volume_Day_lag{day_lag:02d}'] = lagged_values.values

In [42]:
df_main.drop(columns=[
    'Open', 'High', 'Low', '%Chg', 'High-Low', 'High-Low%', 'Last-Open', 'Last-Open%',
       'Roll_SD_Last_10', 'Roll_SDSD_Last_10', 'Roll_SD_Volume_10', 'Change', 'Roll_SDSD_Volume_10', 'ti'
       ], inplace=True)

In [43]:
windows=[4,8,16,26]
vars=['Last', 'Volume', 'Change_Last', 'Change_Volume',
       'Change_High-Low', 'Change_Last-Open', 'Change_Roll_SD_Last_10',
       'Change_Roll_SDSD_Last_10', 'Change_Roll_SD_Volume_10',
       'Change_Roll_SDSD_Volume_10']

# Moving averages
for window in windows:
    for var in vars:
        df_main[f'{var}_ma{window:02d}'] = df_main.groupby('ticker')[var].transform(lambda x: x.rolling(window=window).mean())

In [44]:
# Calculate lags
vars = ['Change_High-Low','Change_High-Low_ma04','Change_High-Low_ma08','Change_High-Low_ma16','Change_High-Low_ma26','Change_Last-Open','Change_Last-Open_ma04','Change_Last-Open_ma08','Change_Last-Open_ma16','Change_Last-Open_ma26','Change_Last_ma04','Change_Last_ma08','Change_Last_ma16','Change_Last_ma26','Change_Roll_SDSD_Last_10','Change_Roll_SDSD_Last_10_ma04','Change_Roll_SDSD_Last_10_ma08','Change_Roll_SDSD_Last_10_ma16','Change_Roll_SDSD_Last_10_ma26','Change_Roll_SDSD_Volume_10','Change_Roll_SDSD_Volume_10_ma04','Change_Roll_SDSD_Volume_10_ma08','Change_Roll_SDSD_Volume_10_ma16','Change_Roll_SDSD_Volume_10_ma26','Change_Roll_SD_Last_10','Change_Roll_SD_Last_10_ma04','Change_Roll_SD_Last_10_ma08','Change_Roll_SD_Last_10_ma16','Change_Roll_SD_Last_10_ma26','Change_Roll_SD_Volume_10','Change_Roll_SD_Volume_10_ma04','Change_Roll_SD_Volume_10_ma08','Change_Roll_SD_Volume_10_ma16','Change_Roll_SD_Volume_10_ma26','Change_Volume_ma04','Change_Volume_ma08','Change_Volume_ma16','Change_Volume_ma26','Last_ma04','Last_ma08','Last_ma16','Last_ma26','Volume_ma04','Volume_ma08','Volume_ma16','Volume_ma26']

for var in vars:
    df_main[f'{var}_lag01'] = df_main.groupby('ticker')[var].shift(1)

vars=['Last', 'Volume', 'Change_Last', 'Change_Volume']

for l in range(1, 27):
    for var in vars:
        df_main[f'{var}_lag{l:02d}'] = df_main.groupby('ticker')[var].shift(l)

In [45]:
# reshape the data to wide format, adding the ticker as a prefix to all columns
to_reshape = df_main.columns.difference(['Time', 'ticker'])
df_wide = df_main.pivot_table(index='Time', columns='ticker', values=to_reshape, aggfunc='first')
# Forward fill missing values
df_wide = df_wide.ffill()

In [46]:
# Flatten the MultiIndex column names
df_wide.columns = [f'{ticker}_{col}' for col, ticker in df_wide.columns]
df_wide = df_wide[df_wide.index >= datetime.datetime(2018, 1, 1, 0, 0)]

In [47]:
# Sort columns alphabetically
df_wide = df_wide.reindex(sorted(df_wide.columns), axis=1)

In [48]:
# Create day of week dummies
df_wide['day_of_week']   = df_wide.index.dayofweek
df_wide['month_of_year'] = df_wide.index.month
df_wide['hour_of_day']   = df_wide.index.hour

# Create dummy variables for categorical features
df_wide = pd.get_dummies(df_wide, columns=['day_of_week', 'month_of_year', 'hour_of_day'], drop_first=False)

df_wide['is_open']  = np.where((df_wide.index.hour == 9)  & (df_wide.index.minute == 30), 1, 0)
df_wide['is_close'] = np.where((df_wide.index.hour == 15) & (df_wide.index.minute == 45), 1, 0)

In [49]:
# Save the processed DataFrame to a pickle file
output_path = r"../../data/processed/stock_data.pkl"
with open(output_path, 'wb') as f:
    pickle.dump(df_wide, f)

In [50]:
df_main = df_main.dropna()
# Create day of week dummies
df_main['day_of_week']   = df_main['Time'].dt.dayofweek
df_main['month_of_year'] = df_main['Time'].dt.month
df_main['hour_of_day']   = df_main['Time'].dt.hour

# Create dummy variables for categorical features
df_main = pd.get_dummies(df_main, columns=['day_of_week', 'month_of_year', 'hour_of_day'], drop_first=False)

df_main['is_open']  = np.where((df_main['Time'].dt.hour == 9)  & (df_main['Time'].dt.minute == 30), 1, 0)
df_main['is_close'] = np.where((df_main['Time'].dt.hour == 15) & (df_main['Time'].dt.minute == 45), 1, 0)

In [51]:
df_wide = df_wide[[i for i in df_wide.columns if any(j in i for j in ['BNO', 'JETS', 'IYT', 'ITA'])]]

In [52]:
df_main = pd.merge(df_main, df_wide, on='Time', how='inner')
df_main.sort_values(by=['Time', 'ticker'], inplace=True)
df_main.index = df_main[['Time','ticker']]
df_main['date'] = df_main['Time']
df_main.drop(columns=['Time'], inplace=True)

In [53]:
df_main = df_main[df_main['ticker'].isin(['AAL', 'ALGT', 'ALK', 'DAL', 'JBLU', 'LUV', 'UAL'])]

In [54]:
# Save the processed DataFrame to a pickle file
output_path = r"../../data/processed/stock_data_long.pkl"
with open(output_path, 'wb') as f:
    pickle.dump(df_main, f)

In [55]:
df_main['ticker'].value_counts()

ticker
DAL     48275
LUV     48274
AAL     48273
JBLU    48273
ALK     48273
UAL     48273
ALGT    48154
Name: count, dtype: int64

In [56]:
df_main[df_main['ticker'] == 'ALGT']['date'].unique()

<DatetimeArray>
['2018-01-02 09:30:00', '2018-01-02 09:45:00', '2018-01-02 10:00:00',
 '2018-01-02 10:15:00', '2018-01-02 10:30:00', '2018-01-02 10:45:00',
 '2018-01-02 11:00:00', '2018-01-02 11:15:00', '2018-01-02 11:30:00',
 '2018-01-02 11:45:00',
 ...
 '2025-05-30 13:30:00', '2025-05-30 13:45:00', '2025-05-30 14:00:00',
 '2025-05-30 14:15:00', '2025-05-30 14:30:00', '2025-05-30 14:45:00',
 '2025-05-30 15:00:00', '2025-05-30 15:15:00', '2025-05-30 15:30:00',
 '2025-05-30 15:45:00']
Length: 48154, dtype: datetime64[ns]

In [57]:
df_main[df_main['ticker'] == 'DAL']['date'].unique()

<DatetimeArray>
['2018-01-02 09:30:00', '2018-01-02 09:45:00', '2018-01-02 10:00:00',
 '2018-01-02 10:15:00', '2018-01-02 10:30:00', '2018-01-02 10:45:00',
 '2018-01-02 11:00:00', '2018-01-02 11:15:00', '2018-01-02 11:30:00',
 '2018-01-02 11:45:00',
 ...
 '2025-05-30 13:30:00', '2025-05-30 13:45:00', '2025-05-30 14:00:00',
 '2025-05-30 14:15:00', '2025-05-30 14:30:00', '2025-05-30 14:45:00',
 '2025-05-30 15:00:00', '2025-05-30 15:15:00', '2025-05-30 15:30:00',
 '2025-05-30 15:45:00']
Length: 48275, dtype: datetime64[ns]

In [58]:
df_main

,Last,Volume,ticker,Change_Last,Change_Volume,Change_High-Low,Change_Last-Open,Change_Roll_SD_Last_10,Change_Roll_SDSD_Last_10,Change_Roll_SD_Volume_10,...,JETS_Volume_lag26,JETS_Volume_ma04,JETS_Volume_ma04_lag01,JETS_Volume_ma08,JETS_Volume_ma08_lag01,JETS_Volume_ma16,JETS_Volume_ma16_lag01,JETS_Volume_ma26,JETS_Volume_ma26_lag01,date
"(2018-01-02 09:30:00, AAL)",52.110,181986.0,AAL,0.0700,-241224.0,0.2450,0.0800,0.005585,0.001349,1.775563e+03,...,500.0,1571.75,1235.00,5573.50,5480.0,3117.6875,3058.4375,2061.076923,2024.615385,2018-01-02 09:30:00
"(2018-01-02 09:30:00, ALGT)",155.450,3789.0,ALGT,0.8000,-9689.0,5.5450,0.1500,0.163960,0.033626,-7.587094e+01,...,500.0,1571.75,1235.00,5573.50,5480.0,3117.6875,3058.4375,2061.076923,2024.615385,2018-01-02 09:30:00
"(2018-01-02 09:30:00, ALK)",74.470,47871.0,ALK,0.9600,-54076.0,0.4200,-0.6100,0.089942,0.012743,-2.173999e+03,...,500.0,1571.75,1235.00,5573.50,5480.0,3117.6875,3058.4375,2061.076923,2024.615385,2018-01-02 09:30:00
"(2018-01-02 09:30:00, DAL)",56.185,344480.0,DAL,0.1750,-221911.0,0.0900,0.0050,-0.000820,-0.001421,1.118285e+04,...,500.0,1571.75,1235.00,5573.50,5480.0,3117.6875,3058.4375,2061.076923,2024.615385,2018-01-02 09:30:00
"(2018-01-02 09:30:00, JBLU)",22.370,293572.0,JBLU,0.0300,-387808.0,0.1290,-0.0100,0.000235,0.000281,1.488588e+03,...,500.0,1571.75,1235.00,5573.50,5480.0,3117.6875,3058.4375,2061.076923,2024.615385,2018-01-02 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2025-05-30 15:45:00, ALK)",50.910,440444.0,ALK,-0.3400,308639.0,0.2150,0.2000,0.032263,0.001000,9.560724e+04,...,153907.0,912924.25,48853.25,469692.75,37950.0,258929.6250,42477.1250,175237.653846,47463.269231,2025-05-30 15:45:00
"(2025-05-30 15:45:00, DAL)",48.380,1245698.0,DAL,-0.1575,998165.0,0.0850,0.0375,0.001618,0.003603,2.191752e+05,...,153907.0,912924.25,48853.25,469692.75,37950.0,258929.6250,42477.1250,175237.653846,47463.269231,2025-05-30 15:45:00
"(2025-05-30 15:45:00, JBLU)",5.050,8400782.0,JBLU,-0.0500,7415193.0,0.0450,0.0250,0.010365,-0.002789,2.213174e+06,...,153907.0,912924.25,48853.25,469692.75,37950.0,258929.6250,42477.1250,175237.653846,47463.269231,2025-05-30 15:45:00
"(2025-05-30 15:45:00, LUV)",33.370,2026886.0,LUV,-0.1350,1546688.0,0.1476,0.0850,0.000564,0.001398,4.742031e+05,...,153907.0,912924.25,48853.25,469692.75,37950.0,258929.6250,42477.1250,175237.653846,47463.269231,2025-05-30 15:45:00


In [59]:
set(df_main[df_main['ticker'] == 'ALGT']['date'].unique()) - set(df_main[df_main['ticker'] == 'DAL']['date'].unique())

{Timestamp('2020-07-14 14:45:00'), Timestamp('2021-03-03 10:45:00')}

In [60]:
# what values of 'date' are missing for ticker == 'ALGT' compared to 'DAL'
missing_dates = df_main[df_main['ticker'] == 'ALGT']['date'].unique()
missing_dates = set(missing_dates) - set(df_main[df_main['ticker'] == 'DAL']['date'].unique())
missing_dates = sorted(missing_dates)
missing_dates

[Timestamp('2020-07-14 14:45:00'), Timestamp('2021-03-03 10:45:00')]

In [61]:
test = df_main['date'].value_counts()
test

date
2025-05-30 15:45:00    7
2018-01-02 09:30:00    7
2018-01-02 09:45:00    7
2018-01-02 10:00:00    7
2018-01-02 10:15:00    7
                      ..
2021-03-03 10:45:00    4
2020-11-27 15:30:00    3
2020-12-24 15:30:00    3
2019-12-24 15:30:00    3
2019-11-29 15:30:00    3
Name: count, Length: 48277, dtype: int64